In [1]:
!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time 

from google.protobuf.json_format import MessageToDict

mp_draining = mp.solutions.drawing_utils # drawing 
mp_pose = mp.solutions.pose # import pose estimation model

## Calculate stuff

In [2]:
import numpy as np
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] -b[1], c[0]-b[0]) - np.arctan2(a[1] -b[1], a[0] - b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle
        
    return angle

In [3]:
import numpy as np
def calculate_distance(point1, point2):
    distance = ((point1[0] - point2[0])**2 + (point1[1]-point2[1])**2)**(0.5)
    return distance

In [4]:
def bentFinger(finger):
    if (calculate_angle(finger[0], finger[1], finger[3]) <90 and finger[3][1] > finger[0][1]):
        return True
    return False

In [5]:
def isBetween(left, middle, right):
    if (middle[0] > min(left[0], right[0]) and middle[0] < max(left[0], right[0])):
        return True
    return False

In [6]:
def isBetweenVertical(left, middle, right):
    if (middle[1] > min(left[1], right[1]) and middle[1] < max(left[1], right[1])):
        return True
    return False

In [7]:
def isLeft(finger):
    if(finger[3][0] < finger[2][0] and finger[2][0] < finger[1][0] and finger[1][0] < finger[0][0]):
        return True
    return False

In [8]:
def straightFinger(finger):
    if (finger[0][1] > finger[1][1] and finger[1][1] > finger[2][1] and finger[2][1] > finger[3][1]):
        return True
    return False

In [9]:
def straightDown(finger):
    if (finger[0][1] < finger[1][1] and finger[1][1] < finger[2][1] and finger[2][1] < finger[3][1]):
        return True
    return False

In [10]:
def time_letters(letter, change_start):
    if change_start:
        start_time = time.perf_counter()
        change_start = False
    else:
        if time.perf_counter() - start_time > 2:
            text += letter
            change_start = True
    return change_start

In [11]:
def displayText2(text, word):
    cv2.putText(image, text, 
        (int(width*0.5), 300), 
            cv2.FONT_HERSHEY_SIMPLEX, 7, (255, 255, 255), 2, cv2.LINE_AA
                )
    if word == "" or word[len(word)-1] != text:
        word += text
        print(word + "!")
    return word

In [12]:
def displayText(text):
    cv2.putText(image, text, 
        (int(width*0.5), 300), 
            cv2.FONT_HERSHEY_SIMPLEX, 7, (255, 255, 255), 2, cv2.LINE_AA
                )

In [13]:
def pastTime(changeMs):
    then = time.time()
    return     then - changeMs

In [14]:
def currTime():
    now =  time.time()
    return now

In [17]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
add = True
change_start = True
start_time = 0
end_time = 0
word = ""

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
        if cap.isOpened():
            width  = cap.get(3)  # float `width`
            height = cap.get(4)
            print (str(width) + " " + str(height))
        alphabet = [1 for x in range(0, 26)][2]
        print(alphabet)
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                  print("Ignoring empty camera frame.")
                  # If loading a video, use 'break' instead of 'continue'.
                  continue

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.        
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            results = hands.process(image)
            
            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)            
            
            try:
                landmarks = results.hand_landmarks.landmark
                print(landmarks)
            except:
                pass

    


            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
                    
                  
                
                #finger landmarks-----------------------
                #18 - pinky second knuckle, 10-middle second knuckle, 4-thumb tip
                
                #numbered starting from knuckle
                #index[1][0] --> x value of first joint down
                index = []
                middle = []
                ring = []
                thumb = []
                pinky = []
                for mark in range(0,4):
                    thumb.append([hand_landmarks.landmark[mark+1].x, hand_landmarks.landmark[mark+1].y, hand_landmarks.landmark[mark+1].z])
                    index.append([hand_landmarks.landmark[mark+5].x, hand_landmarks.landmark[mark+5].y, hand_landmarks.landmark[mark+5].z])
                    middle.append([hand_landmarks.landmark[mark+9].x, hand_landmarks.landmark[mark+9].y, hand_landmarks.landmark[mark+9].z])
                    ring.append([hand_landmarks.landmark[mark+13].x, hand_landmarks.landmark[mark+13].y, hand_landmarks.landmark[mark+13].z])
                    pinky.append([hand_landmarks.landmark[mark+17].x, hand_landmarks.landmark[mark+17].y, hand_landmarks.landmark[mark+17].z])
            
                #max y coord of finger + max x coord of finger
                maxTip = 0
                minTip = 1000
                for coord in hand_landmarks.landmark[8: 21: 4]:
                    maxTip = max(maxTip, coord.y)
                    minTip = min(minTip, coord.y)
            
                #checking for actions-------------------------
                #ok (index + thumb tip distance r close)
                #if (calculate_distance(index[3], thumb[3]) < 0.03):
                    #displayText("ok")
            
               # now = currTime()
                #past2 = pastTime(0.300)
                #past1 = pastTime1(0.400)
                                    
                #if calculate_distance(index[0], middle[0]) < 0.05 and calculate_distance(middle[0], ring[0]) < 0.03 and calculate_distance(ring[0], pinky[0]) < 0.06:
                if max(calculate_distance(pinky[3], ring[3]), calculate_distance(pinky[3], middle[3]), calculate_distance(pinky[3], index[3])) < 0.05 and pinky[3][1] > middle[1][1]:
                    if calculate_distance(pinky[3], thumb[3]) < 0.05:
                        # if alphabet['o' - 'a'][0] > past1 and alphabet['o' - 'a'][0] < past2:
                        word = displayText2('o', word)    
                    else:
                        word = displayText2('c', word)
                elif (bentFinger(index) and bentFinger(middle) and bentFinger(ring) and bentFinger(pinky)):
                    if (isBetween(middle[1], index[1], thumb[3]) and thumb[3][1] < middle[1][1]):
                        word = displayText2('a', word)
                    elif (isBetween(middle[1], thumb[3], index[1]) and thumb[3][1] < middle[1][1]):
                        word = displayText2('t', word)
                    elif(isBetween(ring[1], thumb[3], middle[1]) and thumb[3][1] < ring[1][1]):
                        word = displayText2('n', word)
                    elif(isBetween(pinky[1], thumb[3], ring[1]) and thumb[3][1] < pinky[1][1]):
                        word = displayText2('m', word)
                    elif(isBetween(pinky[1], thumb[3], index[1]) and thumb[3][1] < middle[2][1] and thumb[3][1] > middle[1][1]):
                        word = displayText2('s', word)
                    elif(isBetween(pinky[1], thumb[3], index[1]) and thumb[3][1] > middle[2][1]):
                        word = displayText2('e', word)
                elif(straightFinger(index) and straightFinger(middle) and straightFinger(ring) and straightFinger(pinky)):
                    if (isBetween(pinky[0], thumb[3], index[0])):
                        word = displayText2('b', word)
                elif(bentFinger(index) and straightFinger(middle) and straightFinger(ring) and straightFinger(pinky)):
                    if (calculate_distance(thumb[3], index[3]) < 0.1):
                        word = displayText2('f', word)
                elif(not bentFinger(index) and bentFinger(middle) and bentFinger(ring) and bentFinger(pinky) and pinky[3][1] > pinky[2][1]):
                    if(index[3][1] > index[1][1] and index[3][1] < middle[1][1]):
                        word = displayText2('x', word)
                    elif (calculate_distance(thumb[3], ring[3]) < 0.08):
                        word = displayText2('d', word)
                    elif(calculate_angle(index[3], thumb[0], thumb[3]) > 30):
                        word = displayText2('l', word)
                elif(not bentFinger(index) and not bentFinger(middle) and bentFinger(ring) and bentFinger(pinky)):
                    if (index[3][0] > middle[3][0]):
                        word = displayText2('r', word)
                    else:
                        if (calculate_angle(index[3], middle[0], middle[3]) > 18):
                            if isBetween(middle[1], thumb[3], index[1]) and thumb[3][1] < middle[0][1]:
                                word = displayText2('k', word)
                            else:
                                word = displayText2('v', word)
                        else:
                            word = displayText2('u', word)
                elif(not bentFinger(index) and not bentFinger(middle) and not bentFinger(ring) and bentFinger(pinky)):
                    if (calculate_angle(index[3], middle[0], ring[3]) > 30):
                         word = displayText2('w', word)
                elif(bentFinger(index) and bentFinger(middle) and bentFinger(ring) and not bentFinger(pinky)):
                    if (calculate_angle(pinky[3], middle[3], thumb[3]) < 20):
                        word = displayText2('i', word)
                    elif (calculate_angle(pinky[3], middle[3],thumb[3]) > 45 and thumb[3][1] < index[1][1]):
                         word = displayText2('y', word)
                elif isLeft(index) and isLeft(middle) and straightDown(middle) and isBetweenVertical(index[2], thumb[3], middle[2]) and ring[3][0] > ring[1][0] and pinky[3][0] > pinky[1][0]:
                    word = displayText2('p', word)
                elif isLeft(index) and isLeft(middle) and ring[3][0] > ring[1][0] and pinky[3][0] > pinky[1][0] and calculate_distance(thumb[3], ring[1]) < 0.5:
                    word = displayText2('h', word)
                elif isLeft(index) and isLeft(thumb) and middle[3][0] > middle[1][0] and ring[3][0] > ring[1][0] and pinky[3][0] > pinky[1][0]:
                    if straightDown(index):
                        word = displayText2('q', word)
                    else:
                        word = displayText2('g', word)

                
                if len(results.multi_handedness) == 2:                       
                    word = word[0:-1]
                else:
                    for i in results.multi_handedness:
                        label = MessageToDict(i)[
                        'classification'][0]['label']

                        if label == 'Left':
                            #word = ""
                            #add = True
                            #displayText("left")
                            pass
                        if label == 'Right':
                            #word = ""
                            pass
                
            # Flip the image horizontally for a selfie-view display.
            
            
            cv2.putText(image, word, 
                    (10, 100), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.imshow('Handi', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
cap.release
cv2.destroyAllWindows()

640.0 480.0
1
c!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
ac!
aca!
acac!
acaca!
acacac!
acacaca!
acacacac!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
acacacaca!
aca!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
a!
au!
auc!
auc!
auc!


In [160]:
print(calculate_distance)

<function calculate_distance at 0x00000258C44ED0D0>


In [12]:
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
print(arr[0:-1:2])

[1, 3, 5, 7, 9]
